In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pandas as pd

import numpy as np
import pandas as pd

import seaborn as sns
from statsmodels.tsa.stattools import adfuller, acf, pacf, ccf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import re
from sklearn.linear_model import LinearRegression

In [2]:
import numpy as np
import pandas as pd
import re
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [3]:
def similarity_cosine(vec1, vec2):
    cosine_similarity_arr = []
    for v1,v2 in zip(vec1, vec2):
        cosine_similarity = np.dot(v1, v2)/(np.linalg.norm(v1)* np.linalg.norm(v2))
        cosine_similarity_arr.append(cosine_similarity)
    return np.array(cosine_similarity_arr)

def keep_words_with_underscore(input_string):
    # Define a regular expression pattern to match words with underscores
    pattern = r'\b\w*_[\w_]*\b'

    # Use re.findall to extract words that match the pattern
    matching_words = re.findall(pattern, input_string)

    # Join the matching words to form the final string
    result = ' '.join(matching_words)

    return result


def update_co_occurrences(word_year_list,word_co_occurrences):
    # Iterate through the words in the list
    word_list, year = word_year_list
    
    for word in word_list:
        # If the word is not already in the dictionary, add it with an empty list
        if word not in word_co_occurrences:
            word_co_occurrences[word] = {}
        
        # Add words from the list to the co-occurrence list for the current word
        for other_word in word_list:
            # if other_word != word and other_word not in word_co_occurrences[word]:
            #     word_co_occurrences[word].append(other_word)
            if other_word != word and other_word not in word_co_occurrences[word]:
                word_co_occurrences[word][other_word] = [year] 
            
            elif other_word != word and other_word in word_co_occurrences[word]:
                # word_co_occurrences[word][other_word][0] +=1
                word_co_occurrences[word][other_word].append(year)

In [4]:
concept_filtered_arr = np.load("files/overlapping_filtered_concepts.npy")
ngram_abstracts = np.load("files/ngram_abstracts.npy", mmap_mode="r")
saved_year_arr = np.load("files/year_arr.npy", mmap_mode="r")

print("Concepts which were tracked",concept_filtered_arr.shape)
print("Abstracts",ngram_abstracts.shape)
print("Year associated to abstract",saved_year_arr.shape)

phys_filtered_concept_dict = {k:1 for k in concept_filtered_arr}
ocurr_arr = []
for abstract, year in zip(ngram_abstracts, saved_year_arr):
    temp = keep_words_with_underscore(abstract)
    if temp.count(" ") > 0:
        temp = temp.split(" ") 
        temp = [s for s in temp if s in phys_filtered_concept_dict]
        ocurr_arr.append([list(filter(("_").__ne__, temp)),year])
                        
word_co_occurrences = {}

for word_list in tqdm(ocurr_arr):
    update_co_occurrences(word_list,word_co_occurrences)

# Concepts which cooccur after 2021 
def get_co_occur_concept_pair_after_year_arr(word_co_occurrences, first_occ_year, final_occ_year):
    cnt = 0 
    co_occur_concept_pair_arr = []
    for concept, v in word_co_occurrences.items():
        for co_concept, years in v.items():
            if np.min(years) >= first_occ_year:
                co_occur_concept_pair_arr.append([concept,co_concept])
                cnt += 1 


    return np.array(co_occur_concept_pair_arr)

# example 
print("Pairs of concepts which co-occur after 2021", get_co_occur_concept_pair_after_year_arr(word_co_occurrences, 2021,2023).shape)

Concepts which were tracked (12770,)
Abstracts (157821,)
Year associated to abstract (157821,)


100%|██████████| 152310/152310 [00:07<00:00, 21155.17it/s]


Pairs of concepts which co-occur after 2021 (889874, 2)


In [30]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import CosineAnnealingLR
import numpy as np

class TimeSeriesDataset(Dataset):
    def __init__(self, data, word_co_occurrences, year_arr, c_inx_arr, input_window_size = 5, output_window_size = 3, offset_to_current_year = 1):
        self.train_window_data = data[:,-input_window_size-output_window_size-offset_to_current_year:-output_window_size-offset_to_current_year]
        if offset_to_current_year == 0:
            # self.label_window_data = data[:,-output_window_size:]
            self.label_year_range = year_arr[-output_window_size:]
        else: 
            # self.label_window_data = data[:,-output_window_size-offset_to_current_year:-offset_to_current_year]
            self.label_year_range = year_arr[-output_window_size-offset_to_current_year:-offset_to_current_year]

        
        self.co_occur_concept_pair_arr = get_co_occur_concept_pair_after_year_arr(word_co_occurrences, first_occ_year=self.label_year_range[0], final_occ_year=self.label_year_range[-1])
        self.c_inx_arr = c_inx_arr
        self.input_window_size = input_window_size 
        self.output_window_size = output_window_size 
        self.offset_to_current_year = offset_to_current_year 

    def __len__(self):
        return 64*500

    def __getitem__(self, idx):
        
        if np.random.rand() < 0.5:
            return self._get_positive_sample()
        else:
            return self._get_negative_sample()
    

    def _get_positive_sample(self):
        while True:
            sampled_pairs = np.random.choice(len(self.co_occur_concept_pair_arr), size=1)
            c_pair = self.co_occur_concept_pair_arr[sampled_pairs][0]
            inx_0 = np.where(self.c_inx_arr == c_pair[0])[0]
            inx_1 = np.where(self.c_inx_arr == c_pair[1])[0]
            if inx_0.size > 0 and inx_1.size > 0:
                break
        enc_0 = self.train_window_data[inx_0][0]
        enc_1 = self.train_window_data[inx_1][0]
        enc_01 = np.concatenate((enc_0, enc_1), axis=-1)
        return torch.from_numpy(enc_01), torch.ones(1), torch.from_numpy(np.array([inx_0,inx_1]))

    def _get_negative_sample(self):
        while True:
            sampled_pair = np.random.choice(self.train_window_data.shape[0], size=2)
            if self.c_inx_arr[sampled_pair[1]] not in word_co_occurrences[self.c_inx_arr[sampled_pair[0]]]:
                break
        inx_0 = np.where(self.c_inx_arr == self.c_inx_arr[sampled_pair[0]])[0]
        inx_1 = np.where(self.c_inx_arr == self.c_inx_arr[sampled_pair[1]])[0]
        enc_0 = self.train_window_data[inx_0][0]
        enc_1 = self.train_window_data[inx_1][0]
        enc_01 = np.concatenate((enc_0, enc_1), axis=-1)
        return torch.from_numpy(enc_01), torch.zeros(1), torch.from_numpy(np.array([inx_0,inx_1]))
    
    def _check_indexing(self):
        if self.offset_to_current_year != 0 :
            print(f"... {np.unique(saved_year_arr)[-self.input_window_size-self.output_window_size-self.offset_to_current_year-3:-self.input_window_size-self.output_window_size-self.offset_to_current_year]}",f" -> Training Window {np.unique(saved_year_arr)[-self.input_window_size-self.output_window_size-self.offset_to_current_year:-self.output_window_size-self.offset_to_current_year]}",f" <- {np.unique(saved_year_arr)[-self.output_window_size-self.offset_to_current_year:]}")
            print(f"... {np.unique(saved_year_arr)[-self.output_window_size-self.offset_to_current_year-3:-self.output_window_size-self.offset_to_current_year]}",f" -> Label Window {np.unique(saved_year_arr)[-self.output_window_size-self.offset_to_current_year:-self.offset_to_current_year]}",f" <- {np.unique(saved_year_arr)[-self.offset_to_current_year:]}")
        else:
            print(f"... {np.unique(saved_year_arr)[-self.input_window_size-self.output_window_size-3:-self.input_window_size-self.output_window_size]}",f" -> Training Window {np.unique(saved_year_arr)[-self.input_window_size-self.output_window_size:-self.output_window_size]}",f" <- {np.unique(saved_year_arr)[-self.output_window_size:]}")
            print(f"... {np.unique(saved_year_arr)[-self.output_window_size-3:-self.output_window_size]}",f" -> Label Window {np.unique(saved_year_arr)[-self.output_window_size:]}",f" <- {[]}")
        
class NovelSeriesDataset(Dataset):
    def __init__(self, data, c_inx_arr, input_window_size = 5):
        self.train_window_data = data[:,-input_window_size:]
        
        self.c_inx_arr = c_inx_arr
        self.input_window_size = input_window_size 
        

    def __len__(self):
        return 64*500

    def __getitem__(self, idx):
        
        while True:
            sampled_pair = np.random.choice(self.train_window_data.shape[0], size=2)
            if self.c_inx_arr[sampled_pair[1]] not in word_co_occurrences[self.c_inx_arr[sampled_pair[0]]]:
                break
        inx_0 = np.where(self.c_inx_arr == self.c_inx_arr[sampled_pair[0]])[0]
        inx_1 = np.where(self.c_inx_arr == self.c_inx_arr[sampled_pair[1]])[0]
        enc_0 = self.train_window_data[inx_0][0]
        enc_1 = self.train_window_data[inx_1][0]
        enc_01 = np.concatenate((enc_0, enc_1), axis=-1)
        return torch.from_numpy(enc_01), torch.from_numpy(np.array([inx_0,inx_1]))
    
    def _check_indexing(self):
        
        print(f"... {np.unique(saved_year_arr)[-self.input_window_size-3:-self.input_window_size]}",f" -> Training Window {np.unique(saved_year_arr)[-self.input_window_size:]}",f" <- {[]}")
        


   

# Example usage:
num_samples_per_class = 32
num_features = 128
seq_length = 5
batch_size = 128

encoding_dat = np.load("c_encoding_arr.npy")
c_inx_arr = np.load("c_inx_arr.npy")
print("Representation Vectors for tracked concepts",encoding_dat.shape)
print("Concept associted with representation", c_inx_arr.shape)
scaler = RobustScaler()
reshaped_data = encoding_dat.reshape(-1, encoding_dat.shape[-1])  # Shape: (10000*31, 128)
normalized_data = scaler.fit_transform(reshaped_data)
encoding_data = normalized_data.reshape(encoding_dat.shape)

dataset = TimeSeriesDataset(data=encoding_data, word_co_occurrences=word_co_occurrences, year_arr=np.unique(saved_year_arr), 
                            c_inx_arr=c_inx_arr, input_window_size = 10, output_window_size = 3, offset_to_current_year = 3)
dataset._check_indexing()
print()
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

testing_dataset = TimeSeriesDataset(data=encoding_data, word_co_occurrences=word_co_occurrences, year_arr=np.unique(saved_year_arr), 
                            c_inx_arr=c_inx_arr, input_window_size = 10, output_window_size = 3, offset_to_current_year = 0)
testing_dataset._check_indexing()
testing_dataloader = DataLoader(testing_dataset, batch_size=batch_size, shuffle=True)


print()
novel_dataset = NovelSeriesDataset(data=encoding_data, c_inx_arr=c_inx_arr, input_window_size = 10)
novel_dataset._check_indexing()
novel_dataloader = DataLoader(novel_dataset, batch_size=batch_size, shuffle=True)



Representation Vectors for tracked concepts (12368, 31, 128)
Concept associted with representation (12368,)
... [2006 2007 2008]  -> Training Window [2009 2010 2011 2012 2013 2014 2015 2016 2017 2018]  <- [2019 2020 2021 2022 2023 2024]
... [2016 2017 2018]  -> Label Window [2019 2020 2021]  <- [2022 2023 2024]

... [2009 2010 2011]  -> Training Window [2012 2013 2014 2015 2016 2017 2018 2019 2020 2021]  <- [2022 2023 2024]
... [2019 2020 2021]  -> Label Window [2022 2023 2024]  <- []

... [2012 2013 2014]  -> Training Window [2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]  <- []


In [31]:
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x
    
# Define the model, loss function, optimizer, and scheduler
input_dim = dataset.train_window_data.shape[1] * dataset.train_window_data.shape[2] * 2  # Flattened size * 2 for concatenated pairs
model = MLP(input_dim=input_dim)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = CosineAnnealingLR(optimizer, T_max=10)

# Training the model with early stopping
num_epochs = 50
patience = 5
best_val_loss = float('inf')
early_stopping_counter = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    
    for data, labels, _ in train_loader:
        data = data.view(data.size(0), -1).float()  # Flatten the input data
        labels = labels.float()
        
        optimizer.zero_grad()
        outputs = model(data)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        predicted = (outputs > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()
    
    scheduler.step()
    train_accuracy = 100 * correct_train / total_train
    train_loss = running_loss / len(train_loader)

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for data, labels, _ in val_loader:
            data = data.view(data.size(0), -1).float()  # Flatten the input data
            labels = labels.float()
            outputs = model(data)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            predicted = (outputs > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()
    
    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, '
          f'Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("Early stopping triggered")
            break

Epoch [1/50], Train Loss: 0.6096, Train Accuracy: 65.98%, Val Loss: 0.5685, Val Accuracy: 71.39%
Epoch [2/50], Train Loss: 0.5494, Train Accuracy: 73.30%, Val Loss: 0.5295, Val Accuracy: 75.03%
Epoch [3/50], Train Loss: 0.5318, Train Accuracy: 74.49%, Val Loss: 0.5194, Val Accuracy: 74.83%
Epoch [4/50], Train Loss: 0.5242, Train Accuracy: 74.78%, Val Loss: 0.5320, Val Accuracy: 74.64%
Epoch [5/50], Train Loss: 0.5143, Train Accuracy: 75.38%, Val Loss: 0.5162, Val Accuracy: 75.20%
Epoch [6/50], Train Loss: 0.5014, Train Accuracy: 76.34%, Val Loss: 0.5112, Val Accuracy: 75.38%
Epoch [7/50], Train Loss: 0.4929, Train Accuracy: 76.71%, Val Loss: 0.5055, Val Accuracy: 75.23%
Epoch [8/50], Train Loss: 0.4963, Train Accuracy: 76.38%, Val Loss: 0.4894, Val Accuracy: 76.72%
Epoch [9/50], Train Loss: 0.4968, Train Accuracy: 76.28%, Val Loss: 0.4849, Val Accuracy: 77.61%
Epoch [10/50], Train Loss: 0.4969, Train Accuracy: 76.13%, Val Loss: 0.4873, Val Accuracy: 77.05%
Epoch [11/50], Train Loss: 0.

In [36]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on the validation set
model.eval()
correct_val = 0
total_val = 0

indices = []
outputs_list = []
correct_indices = []
labels_list = []

x = np.arange(31).reshape(-1, 1)
lin_model = LinearRegression()

with torch.no_grad():
    for data, labels, inx in testing_dataloader:
        data = data.view(data.size(0), -1).float()  # Flatten the input data
        labels = labels.float()
        outputs = model(data)
        predicted = (outputs > 0.5).float()
        total_val += labels.size(0)
        correct_val += (predicted == labels).sum().item()
        
        # Collect indices, outputs, labels, and correct predictions
        indices.extend(inx.cpu().numpy())
        outputs_list.extend(outputs.cpu().numpy())
        labels_list.extend(labels.cpu().numpy())
        correct_indices.extend((predicted == labels).cpu().numpy())
        

# Convert lists to numpy arrays for sorting
indices = np.array(indices)
outputs_list = np.array(outputs_list).flatten()
labels_list = np.array(labels_list).flatten()
correct_indices = np.array(correct_indices).flatten()

# Get sorted indices of the outputs
sorted_indices = np.argsort(outputs_list)

# Separate the indices of correct predictions into two categories
correct_0 = []
correct_1 = []

for i in sorted_indices:
    if correct_indices[i]:
        if labels_list[i] == 0:
            correct_0.append(indices[i])
        else:
            correct_1.append(indices[i])

# Print indices of correct predictions
print("Correct predictions to have no co-occurance:")
for cnt,idx in enumerate(correct_0):
    sim = similarity_cosine(encoding_dat[idx[0]][0],encoding_dat[idx[1]][0])
    lin_model.fit(x, sim.reshape(-1, 1))
    slope = lin_model.coef_[0][0]
    print(c_inx_arr[idx[0]],c_inx_arr[idx[1]], np.round(slope,3))
    if cnt ==5:
        break

print("\n Correct predictions to have co-occurance:")
for cnt,idx in enumerate(correct_1):
    sim = similarity_cosine(encoding_dat[idx[0]][0],encoding_dat[idx[1]][0])
    lin_model.fit(x, sim.reshape(-1, 1))
    slope = lin_model.coef_[0][0]
    print(c_inx_arr[idx[0]],c_inx_arr[idx[1]], np.round(slope,3))
    if cnt ==5:
        break

print(f"\nValidation Accuracy: {100 * correct_val / total_val:.2f}%")

Correct predictions to have no co-occurance:
['rayleigh_scattering'] ['abelian_group'] 0.002
['spatial_dependence'] ['entanglement_transformation'] -0.005
['nitrogen_vacancy_center'] ['information_causality'] -0.01
['probabilistic_finite_automaton'] ['impurity_spin'] 0.0
['quantum_interactive_proof'] ['analog_model'] -0.014
['message_passing'] ['electron_vortex'] -0.003

 Correct predictions to have co-occurance:
['cold_atom_experiment'] ['ultra_high_vacuum'] 0.006
['perfect_tensor'] ['tensor_decomposition'] 0.016
['general_solution'] ['variational_problem'] 0.003
['scale_invariant'] ['metal_insulator_transition'] 0.009
['natural_language'] ['effective_field_theory'] 0.009
['sign_problem'] ['convex_cone'] -0.002

Validation Accuracy: 75.43%


In [37]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation on the validation set
model.eval()
correct_val = 0
total_val = 0

indices = []
outputs_list = []
predicted_list = []

with torch.no_grad():
    for data, inx in novel_dataloader:
        data = data.view(data.size(0), -1).float()  # Flatten the input data
        
        outputs = model(data)
        predicted = (outputs > 0.5).float()
        
        # Collect indices, outputs, labels, and correct predictions
        indices.extend(inx.cpu().numpy())
        outputs_list.extend(outputs.cpu().numpy())
        predicted_list.extend(predicted.cpu().numpy())
        
# Convert lists to numpy arrays for sorting
indices = np.array(indices)
outputs_list = np.array(outputs_list).flatten()
predicted_list = np.array(predicted_list).flatten()


# Get sorted indices of the outputs
sorted_indices = np.argsort(outputs_list)

# Separate the indices of correct predictions into two categories
correct_0 = []
correct_1 = []

for i in sorted_indices:
    if predicted_list[i]:
        correct_1.append(indices[i])
    else:
        correct_0.append(indices[i])

# Print indices of correct predictions
print(" Predictions to have no co-occurance:")
for cnt,idx in enumerate(correct_0):
    sim = similarity_cosine(encoding_dat[idx[0]][0],encoding_dat[idx[1]][0])
    lin_model.fit(x, sim.reshape(-1, 1))
    slope = lin_model.coef_[0][0]
    print(c_inx_arr[idx[0]],c_inx_arr[idx[1]], np.round(slope,3))
    if cnt ==5:
        break

print("\n Predictions to have co-occurance:")
for cnt,idx in enumerate(correct_1):
    sim = similarity_cosine(encoding_dat[idx[0]][0],encoding_dat[idx[1]][0])
    lin_model.fit(x, sim.reshape(-1, 1))
    slope = lin_model.coef_[0][0]
    print(c_inx_arr[idx[0]],c_inx_arr[idx[1]],np.round(slope,3))
    if cnt ==5:
        break

 Predictions to have no co-occurance:
['promise_problem'] ['photon_radiation'] -0.011
['jones_polynomial'] ['superradiant_emission'] -0.003
['quantum_adiabatic_algorithm'] ['excited_state_lifetime'] -0.002
['quantum_adiabatic_algorithm'] ['rydberg_excitons'] -0.001
['spin_orientation'] ['quantum_secret_sharing_scheme'] -0.001
['quantum_pcp_conjecture'] ['reflected_signal'] 0.004

 Predictions to have co-occurance:
['long_range_anisotropic_interaction'] ['deep_strong_coupling'] -0.0
['time_optimal_control'] ['harmonic_oscillator_mode'] 0.009
['transfer_protocol'] ['variational_quantum_state'] -0.001
['zitterbewegung_effect'] ['circular_ring'] -0.001
['electric_field_fluctuation'] ['relaxation_oscillation'] -0.007
['particle_number_fluctuation'] ['spinless_particle'] 0.008


In [14]:
stop

NameError: name 'stop' is not defined

In [ ]:
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from tqdm import tqdm
# from concurrent.futures import ProcessPoolExecutor

# def similarity_cosine(vec1, vec2):
#     # Compute cosine similarity in a vectorized manner
#     dot_product = np.sum(vec1 * vec2, axis=1)
#     norm1 = np.linalg.norm(vec1, axis=1)
#     norm2 = np.linalg.norm(vec2, axis=1)
#     cosine_similarity = dot_product / (norm1 * norm2)
#     return cosine_similarity

# def compute_similarity_and_slope(args):
#     cnt_0, cnt_1, label_window_data, threshold, x = args
#     label_subset_0 = label_window_data[cnt_0]
#     label_subset_1 = label_window_data[cnt_1]

#     sim = similarity_cosine(label_subset_0, label_subset_1)

#     model.fit(x, sim.reshape(-1, 1))
#     slope = model.coef_[0][0]

#     if slope < -threshold:
#         return [cnt_0, cnt_1], 0
#     elif slope > threshold:
#         return [cnt_0, cnt_1], 1
#     return None, None

# # label_window_data = np.random.rand(10000, 3, 128)  # Replace this with your actual data
# x = np.arange(3).reshape(-1, 1)
# model = LinearRegression()
# threshold = 0.1

# # Use ProcessPoolExecutor for parallel processing
# executor = ProcessPoolExecutor(max_workers=4)

# tasks = [(cnt_0, cnt_1, label_window_data, threshold, x) for cnt_0 in range(label_window_data.shape[0])
#          for cnt_1 in range(cnt_0 + 1, label_window_data.shape[0])]

# results = list(tqdm(executor.map(compute_similarity_and_slope, tasks), total=len(tasks)))

# idx_pair_0_arr = [result[0] for result in results if result[1] == 0]
# idx_pair_1_arr = [result[0] for result in results if result[1] == 1]

# print("idx_pair_0_arr:", idx_pair_0_arr)
# print("idx_pair_1_arr:", idx_pair_1_arr)


In [38]:
# import numpy as np
# from sklearn.linear_model import LinearRegression
# from tqdm import tqdm

# def similarity_cosine(vec1, vec2):
#     # Compute cosine similarity in a vectorized manner
#     dot_product = np.sum(vec1 * vec2, axis=1)
#     norm1 = np.linalg.norm(vec1, axis=1)
#     norm2 = np.linalg.norm(vec2, axis=1)
#     cosine_similarity = dot_product / (norm1 * norm2)
#     return cosine_similarity


# x = np.arange(3).reshape(-1, 1)
# model = LinearRegression()
# threshold = 0.1

# idx_pair_0_arr = []
# idx_pair_1_arr = []

# # Iterate over pairs in an optimized manner
# for cnt_0 in tqdm(range(label_window_data.shape[0])):
#     for cnt_1 in range(cnt_0 + 1, label_window_data.shape[0]):
#         label_subset_0 = label_window_data[cnt_0]
#         label_subset_1 = label_window_data[cnt_1]

#         sim = similarity_cosine(label_subset_0, label_subset_1)

#         model.fit(x, sim.reshape(-1, 1))
#         slope = model.coef_[0][0]

#         if slope < -threshold:
#             idx_pair_0_arr.append([cnt_0, cnt_1])
#         elif slope > threshold:
#             idx_pair_1_arr.append([cnt_0, cnt_1])

# idx_pair_0_arr = np.array(idx_pair_0_arr)
# idx_pair_1_arr = np.array(idx_pair_1_arr)

# print("idx_pair_0_arr:", idx_pair_0_arr)
# print("idx_pair_1_arr:", idx_pair_1_arr)


In [39]:
# input_window_size = 5  # Number of input time steps (e.g., 0-25)
# output_window_size = 3  # Number of time steps to predict (e.g., 25-28)
# offset_to_current_year = 1 

# # Assuming your data is in a NumPy array of shape (10000, 31, 128)
# data = np.load("c_encoding_arr.npy")
# c_inx_arr = np.load("c_inx_arr.npy")

# # Ensure the total number of time steps is enough to create the windows
# total_time_steps = data.shape[1]

# train_window_data = data[:,-input_window_size-output_window_size-offset_to_current_year:-output_window_size-offset_to_current_year]
# label_window_data = data[:,-output_window_size-offset_to_current_year:-offset_to_current_year]



# # Label 0 => Similarity Decreasing
# # Label 1 => Similarity Increasing 
# # Linear Fit as threshhold 


# from sklearn.linear_model import LinearRegression
# def similarity_cosine(vec1, vec2):
#     cosine_similarity_arr = []
#     for v1,v2 in zip(vec1, vec2):
#         cosine_similarity = np.dot(v1, v2)/(np.linalg.norm(v1)* np.linalg.norm(v2))
#         cosine_similarity_arr.append(cosine_similarity)
#     return np.array(cosine_similarity_arr)

# x = np.arange(3).reshape(-1, 1)
# model = LinearRegression()

# idx_pair_0_arr = []
# idx_pair_1_arr = []
# threshhold = 0.1
# for cnt_0, label_subset_0 in tqdm(enumerate(label_window_data)):
#     for cnt_1,label_subset_1 in enumerate(label_window_data[cnt_0+1:]):
#         sim = similarity_cosine(label_subset_0, label_subset_1)

#         model.fit(x, sim.reshape(-1, 1))
#         slope = model.coef_[0][0]

#         if slope < -threshhold:
#             idx_pair_0_arr.append([cnt_0,cnt_1])
#         elif slope > threshhold:
#             idx_pair_1_arr.append([cnt_0,cnt_1])

        
#     if cnt_0==50:
#         break
# idx_pair_0_arr = np.array(idx_pair_0_arr)
# idx_pair_1_arr = np.array(idx_pair_1_arr)

# idx_pair_0_arr.shape, idx_pair_1_arr.shape

# # # Normalize the data
# # scaler = RobustScaler()

# # # Reshape data to 2D for normalization
# # reshaped_data = data.reshape(-1, data.shape[-1])  # Shape: (10000*31, 128)
# # normalized_data = scaler.fit_transform(reshaped_data)

# # # Reshape back to original shape
# # normalized_data = normalized_data.reshape(data.shape)

# # # normalized_data = torch.tensor(normalized_data, dtype=torch.float32)

# # train_data = normalized_data[:, :28, :]
# # train_labels = normalized_data[:, 28:, :]

# # Prepare training inputs and outputs
# # X_train = normalized_data[:, :input_window_size, :]  # Shape: (10000, input_window_size, 128)
# # Y_train = normalized_data[:, input_window_size:input_window_size + output_window_size, :]  # Shape: (, output_window_size, 128)

# # # Prepare validation inputs and outputs
# # X_val = normalized_data[:, 1:1 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# # Y_val = normalized_data[:, 1 + input_window_size:1 + input_window_size + output_window_size, :]  # Shape: (, output_window_size, 128)

# # # Prepare tnormalized_dataest inputs and outputs
# # X_test = normalized_data[:, 2:2 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# # Y_test = normalized_data[:, 2 + input_window_size:2 + input_window_size + output_window_size, :]  # Shape: (, output_window_size, 128)

In [40]:
# # Sample data for demonstration
# np.random.seed(42)
# data = normalized_data[:100]  # Replace this with your actual data

# # Reshape data for easier manipulation
# samples, timesteps, features = data.shape
# data_reshaped = data[0]#.reshape(-1, features)

# # Convert to DataFrame for easier analysis
# df = pd.DataFrame(data_reshaped, columns=[f'feature_{i}' for i in range(features)])

# # Function to plot time series
# def plot_time_series(df, title=''):
#     plt.figure(figsize=(15, 6))
#     for column in df.columns:
#         plt.plot(df[column], label=column)
#     plt.title(title)
#     plt.xlabel('Time Steps')
#     plt.ylabel('Value')
#     # plt.legend(loc='upper right')
#     plt.show()

# # Function to plot rolling statistics
# def plot_rolling_statistics(df, window=5):
#     rolling_mean = df.rolling(window=window).mean()
#     rolling_std = df.rolling(window=window).std()
    
#     plt.figure(figsize=(15, 6))
#     plt.plot(df, label='Original')
#     plt.plot(rolling_mean, label='Rolling Mean')
#     plt.plot(rolling_std, label='Rolling Std')
#     plt.legend(loc='upper right')
#     plt.show()

# # Function to perform ADF test
# def adf_test(series):
#     result = adfuller(series)
#     print(f'ADF Statistic: {result[0]}')
#     print(f'p-value: {result[1]}')
#     for key, value in result[4].items():
#         print('Critial Values:')
#         print(f'   {key}, {value}')

# # Function to plot ACF and PACF
# def plot_acf_pacf(series, lags=14):
#     plt.figure(figsize=(15, 6))
#     plt.subplot(121)
#     plt.plot(acf(series, nlags=lags))
#     plt.title('Autocorrelation Function')
#     plt.subplot(122)
#     plt.plot(pacf(series, nlags=lags))
#     plt.title('Partial Autocorrelation Function')
#     plt.show()

# # Function to perform seasonal decomposition
# def decompose_series(series, freq=7):
#     decomposition = seasonal_decompose(series, period=freq)
#     decomposition.plot()
#     plt.show()

# # Perform correlation analysis
# correlation_matrix = df.corr()
# sns.heatmap(correlation_matrix, cmap='coolwarm')
# plt.title('Correlation Matrix')
# plt.show()

# # Visualize data
# plot_time_series(df.head(1000))  # Plot the first 1000 samples

# # ADF test for stationarity (example on the first feature)
# adf_test(df['feature_0'])

# # Plot rolling statistics (example on the first feature)
# plot_rolling_statistics(df['feature_0'])



# # ACF and PACF plots (example on the first feature)
# plot_acf_pacf(df['feature_0'])

# # PCA for dimensionality reduction
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data_reshaped)
# pca = PCA(n_components=2)
# pca_result = pca.fit_transform(data_scaled)

# plt.figure(figsize=(10, 6))
# plt.scatter(pca_result[:, 0], pca_result[:, 1], alpha=0.2)
# plt.title('PCA Results' + str(pca.explained_variance_))
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.show()


In [41]:
# def plot_pred(forecast_arr,max_inx=10000):
#     fig, axs = plt.subplots(3)
#     for i in range(3):
#         inx_c = np.random.randint(max_inx)
#         inx_d = np.random.randint(128)

#         axs[i].plot(train_data[inx_c,:,inx_d])
#         axs[i].plot(np.arange(28,31),forecast_arr[inx_c,:,inx_d],".",label="pred")
#         axs[i].plot(np.arange(28,31),train_labels[inx_c,:,inx_d],label="real")
#     plt.legend()
#     plt.tight_layout()
#     plt.show()

Baseline 1: Naive random walk

In [42]:
# # Simple benchmark: Repeat the last value of the 28th time step for the next 3 steps
# # Extract the last time step from the input sequence
# last_time_step = train_data[:, -1, :] 

# # Repeat the last time step for the next 3 steps
# y_pred = np.tile(last_time_step[:, np.newaxis, :], (1, train_labels.shape[1], 1))  

# # Calculate performance metrics, e.g., Mean Squared Error (MSE)
# mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred.reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,3)}')  

# plot_pred(y_pred)


Baseline 2: Two-Step naive random walk

In [43]:
# # Extract the last two time steps
# second_last_time_step = train_data[:, -5, :]  
# last_time_step = train_data[:, -1, :]         

# # Calculate the trend (difference between the last two time steps)
# trend = last_time_step - second_last_time_step  

# # Use the last value and add the trend for each future time step
# y_pred_informed_random_walk = np.zeros((train_labels.shape[0], train_labels.shape[1], 128))

# for i in range(train_labels.shape[1]):
#     y_pred_informed_random_walk[:, i, :] = last_time_step + (i + 1) * trend

# mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred_informed_random_walk.reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,3)}')
# plot_pred(y_pred_informed_random_walk)

Baseline 3: Continue with historical mean

In [44]:
# mean_time_step = np.mean(train_data[:,-5:,:], axis=1)  # Shape: (10000, 128)
# y_pred_mean = np.tile(mean_time_step[:, np.newaxis, :], (1, train_labels.shape[1], 1))  # Shape: (10000, 5, 128)

# # Calculate performance metrics, e.g., Mean Squared Error (MSE)
# mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred_mean.reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,3)}')
# plot_pred(y_pred_mean)


4. Simple Moving average

In [45]:
# def moving_average_predict(X, window_size=3):
#     # Calculate the moving average of the last 'window_size' time steps
#     moving_avg = np.mean(X[:, -window_size:, :], axis=1)  # Shape: (10000, 128)
    
#     # Repeat the moving average for the next 5 time steps
#     y_pred = np.tile(moving_avg[:, np.newaxis, :], (1, train_labels.shape[1], 1))  # Shape: (10000, 5, 128)
    
#     return y_pred

# # Benchmark: Moving Average

# for window_size in range(0,10):
#     y_pred_moving_avg = moving_average_predict(train_data, window_size)
#     mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred_moving_avg.reshape(-1, train_labels.shape[1] * 128))

#     print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,3)} with window size: {window_size}')

# plot_pred(y_pred_moving_avg)

In [46]:
# def moving_average_diff_predict(X, window_size=3, predict_steps=3):
#     # Calculate the difference between consecutive time steps
    
#     # Calculate the mean of the differences over the window size
#     moving_avg_diff = np.mean(X[:, -window_size:, :], axis=1)  # Shape: (10000, 128)
    
#     # Initialize predictions array
#     y_pred = np.zeros((X.shape[0], predict_steps, X.shape[2]))  # Shape: (10000, 5, 128)
    
#     # Get the last observed time step
#     last_time_step = X[:, -1, :]  # Shape: (10000, 128)
    
#     # Predict the future steps based on the mean difference
#     for i in range(predict_steps):
#         y_pred[:, i, :] = last_time_step + (i + 1) * moving_avg_diff
    
#     return y_pred


# for window_size in range(0,10):
#     y_pred_moving_avg = moving_average_diff_predict(train_data, window_size)
#     mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred_moving_avg.reshape(-1, train_labels.shape[1] * 128))

#     print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)} with window size: {window_size}')

# plot_pred(y_pred_moving_avg)

5. Difference Informed Moving Average

In [47]:
# def moving_average_diff_predict(X, window_size=3, predict_steps=3):
#     # Calculate the difference between consecutive time steps
#     grad = np.diff(X, axis=1)  # Shape: (10000, 25, 128)
    
#     # Calculate the mean of the differences over the window size
#     moving_avg_diff = np.mean(grad[:, -window_size:, :], axis=1)  # Shape: (10000, 128)
    
#     # Initialize predictions array
#     y_pred = np.zeros((X.shape[0], predict_steps, X.shape[2]))  # Shape: (10000, 5, 128)
    
#     # Get the last observed time step
#     last_time_step = X[:, -1, :]  # Shape: (10000, 128)
    
#     # Predict the future steps based on the mean difference
#     for i in range(predict_steps):
#         y_pred[:, i, :] = last_time_step + (i + 1) * moving_avg_diff
    
#     return y_pred


# for window_size in range(0,10):
#     y_pred_moving_avg = moving_average_diff_predict(train_data, window_size)
#     mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128), y_pred_moving_avg.reshape(-1, train_labels.shape[1] * 128))

#     print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)} with window size: {window_size}')
# plot_pred(y_pred_moving_avg)

Statistical Models Univariate

In [48]:
# from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa.statespace.sarimax import SARIMAX
# from statsmodels.tsa.holtwinters import ExponentialSmoothing
# from statsmodels.tsa.holtwinters import Holt
# from statsmodels.tsa.holtwinters import SimpleExpSmoothing
# from statsmodels.tsa.forecasting.theta import ThetaModel
# from statsmodels.tsa.seasonal import STL
# from prophet import Prophet

In [49]:
# # AR Model
# def ar_model(data, steps=3):
#     model = AutoReg(data, lags=3).fit()
#     forecast = model.predict(start=len(data), end=len(data) + steps - 1)
#     # plot_forecast(data, forecast, 'AR Model')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = ar_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==500:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the AR model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

In [ ]:
# from statsmodels.tsa.stattools import adfuller
# import warnings 
# # Ignore warnings
# warnings.filterwarnings("ignore")

# # Function to perform grid search for ARIMA parameters
# def grid_search_arima(data, p_values, d_values, q_values):
#     best_score, best_cfg = float("inf"), None
#     for p in p_values:
#         for d in d_values:
#             for q in q_values:
#                 order = (p, d, q)
#                 try:
#                     model = ARIMA(data, order=order)
#                     model_fit = model.fit()
#                     aic = model_fit.aic
#                     if aic < best_score:
#                         best_score, best_cfg = aic, order
#                     # print(f'ARIMA{order} AIC={aic}')
#                 except:
#                     continue
#     # print(f'Best ARIMA{best_cfg} AIC={best_score}')
#     return best_cfg

# # Define the p, d, q ranges to search
# p_values = range(0, 4)
# d_values = range(0, 3)
# q_values = range(0, 4)

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         best_order = grid_search_arima(conc_enc[:,inx], p_values, d_values, q_values)
#         print(best_order)
#         fcast = arima_model(conc_enc[:,inx],best_order)
#         forecast_arr[c1,:,inx] = fcast
#         print(inx)
#     if c1==1:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the arima model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)


# best_score, best_cfg = float("inf"), None
# for p in tqdm(p_values):
#     for d in d_values:
#         for q in q_values:
#             order = (p, d, q)
#             forecast_arr = np.zeros_like(train_labels)
#             for c1, conc_enc in enumerate(train_data):
#                 for inx in range(128):
#                     fcast = arima_model(conc_enc[:,inx],order)
#                     forecast_arr[c1,:,inx] = fcast
#                 if c1==5:
#                     break

#             mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))
#             if mse < best_score:
#                         best_score, best_cfg = mse, order
# print(f'Best ARIMA{best_cfg} AIC={best_score}')

In [50]:
# # ARIMA Model
# def arima_model(data, order, steps=3):
#     model = ARIMA(data, order=order).fit()
#     forecast = model.forecast(steps=steps)
#     # plot_forecast(data, forecast, 'ARIMA Model')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = arima_model(conc_enc[:,inx],(0,1,0))
#         forecast_arr[c1,:,inx] = fcast
#     if c1==500:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the arima model: {np.round(mse,4)}')
# plot_pred(forecast_arr,max_inx=c1)

In [51]:
# # Simple Exponential Smoothing
# def ses_model(data, steps=3):
#     model = SimpleExpSmoothing(data, initialization_method="heuristic").fit(
#             smoothing_level=0.75, optimized=False 
#         )
#     forecast = model.forecast(steps=steps)
#     # plot_forecast(data, forecast, 'Simple Exponential Smoothing')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = ses_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==500:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

In [52]:
# # Holt’s Linear Trend Model
# def holt_model(data, steps=3):
#     model = Holt(data,initialization_method="heuristic").fit(
#             smoothing_level=0.75, optimized=False, smoothing_trend=0.1
#         )
#     forecast = model.forecast(steps=steps)
#     # plot_forecast(data, forecast, 'Holt’s Linear Trend Model')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = holt_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==500:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

In [53]:
# # Holt-Winters Seasonal Model
# def holt_winters_model(data, steps=3):
#     model = ExponentialSmoothing(data, trend='add', damped_trend=True).fit(
#             smoothing_level=1, smoothing_trend=0.1,damping_trend=0.2,optimized=True
#         )
#     forecast = model.forecast(steps=steps)
#     # plot_forecast(data, forecast, 'Holt-Winters Seasonal Model')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = holt_winters_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==500:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

In [54]:
# import logging
# # Prophet Model
# def prophet_model(data, steps=3):
#     df = pd.DataFrame({'ds': pd.date_range(start='2023-01-01', periods=len(data), freq='D'), 'y': data})
#     model = Prophet(daily_seasonality=False)
#     model.fit(df)
#     future = model.make_future_dataframe(periods=steps)
#     forecast = model.predict(future)
#     forecast_values = forecast['yhat'].iloc[-steps:].values
#     # plot_forecast(data, forecast_values, 'Prophet Model')
#     return forecast_values

# logging.getLogger("cmdstanpy").disabled = True #  turn 'cmdstanpy' logs off

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = prophet_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==10:
#         break
# logging.getLogger("cmdstanpy").disabled = False #  revert original setting

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

In [55]:
# # Theta Model
# def theta_model(data, steps=3):
#     model = ThetaModel(data,period=10).fit()
#     forecast = model.forecast(steps=steps)
#     # plot_forecast(data, forecast, 'Theta Model')
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#     for inx in range(128):
#         fcast = theta_model(conc_enc[:,inx])
#         forecast_arr[c1,:,inx] = fcast
#     if c1==50:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')

# plot_pred(forecast_arr,max_inx=c1)

Statistical Models Multivariate

In [57]:
# def reject_outliers(data, m = 10.):
#     d = np.abs(data - np.median(data))
#     mdev = np.median(d)
#     s = d/mdev if mdev else np.zeros(len(d))

#     data[s>=m]=0
#     return data

In [56]:
# from statsmodels.tsa.api import VAR

# cnt = 0
# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
#   try:
#     model = VAR(conc_enc)
#     model_fit = model.fit(trend="n")
#     forecast_arr[c1,:] = model_fit.forecast(y=conc_enc,steps=3)
#   except:
#     forecast_arr[c1,:] = np.zeros(128)
#     cnt += 1 

# mse = mean_squared_error(train_labels.reshape(-1, train_labels.shape[1] * 128).flatten(), reject_outliers(forecast_arr.reshape(-1, train_labels.shape[1] * 128).flatten()))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')
# plot_pred(forecast_arr)

In [58]:
# from statsmodels.tsa.statespace.dynamic_factor import DynamicFactor

# def dfm_model(df, steps=3):
#     model = DynamicFactor(df, k_factors=1, factor_order=2,enforce_stationarity=False)
#     results = model.fit(disp=False)
#     forecast = results.forecast(steps=steps)
#     return forecast

# forecast_arr = np.zeros_like(train_labels)
# for c1, conc_enc in tqdm(enumerate(train_data)):
# #   try:
    
#     forecast_arr[c1,:] = dfm_model(conc_enc)
# #   except:
# #     forecast_arr[c1,:] = np.zeros(128)
# #     cnt += 1 
#     if c1==5:
#         break

# mse = mean_squared_error(train_labels[:c1].reshape(-1, train_labels.shape[1] * 128), forecast_arr[:c1].reshape(-1, train_labels.shape[1] * 128))

# print(f'Mean Squared Error (MSE) of the benchmark model: {np.round(mse,4)}')
# plot_pred(forecast_arr)

ML Models Univariate

 Linear regression

In [59]:
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error
# kf = KFold(n_splits=5, shuffle=True)

# for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
#     print(f"FOLD {fold+1}")
#     print("--------------------------------")
    
#     # Split data
#     train_fold_data = torch.Tensor(train_data[train_idx].reshape(-1,train_data.shape[1]*train_data.shape[2]))
#     train_fold_labels = torch.Tensor(train_labels[train_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2]))
#     val_fold_data = torch.Tensor(train_data[val_idx].reshape(-1,train_data.shape[1]*train_data.shape[2]))
#     val_fold_labels = torch.Tensor(train_labels[val_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2]))

#     # Model
#     lr = LinearRegression()
#     lr.fit(train_fold_data, train_fold_labels)
#     predictions_train = lr.predict(train_fold_data)
#     predictions_val = lr.predict(val_fold_data)
#     print(mean_squared_error(train_fold_labels,predictions_train))
#     print(mean_squared_error(val_fold_labels,predictions_val))



In [ ]:
# import numpy as np
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error

# # Example data (replace with your actual data)
# train_data = np.random.rand(10, 28, 128)  # Array with shape (10, 28, 128)
# train_labels = np.random.rand(10, 3, 128)  # Array with shape (10, 3, 128)

# for t_d, t_l in zip(train_data, train_labels):
    
#     # Prepare the data with lagged features
#     window_size = 20
#     future_step = 2  # Third step ahead, considering zero-based indexing

#     X_lagged = []
#     y_lagged = []
#     for i in range(len(t_d) - window_size - future_step ):
#         X_lagged.append(t_d[i:i+window_size].reshape(-1))  # Flatten to (20, 128)
#         y_lagged.append(t_d[i+window_size+future_step].reshape(1, -1))  # Reshape to (1, 128)

#     X_lagged = np.array(X_lagged)
#     y_lagged = np.array(y_lagged)

#     # Create DMatrix for XGBoost
#     dtrain = xgb.DMatrix(X_lagged, label=y_lagged)

#     # Define XGBoost parameters
#     params = {
#         'objective': 'reg:squarederror',
#         'eval_metric': 'rmse',
#     }

#     # Train the model
#     num_boost_round = 100
#     bst = xgb.train(params, dtrain, num_boost_round)

#     # Validation
#     last_window = t_d[-window_size:].reshape(1, -1)  # Flatten to (20, 128)
#     dfuture = xgb.DMatrix(last_window)
#     future_pred = bst.predict(dfuture)

#     val_mse = mean_squared_error(future_pred, t_l[future_step].reshape(1, -1))
#     print("Validation MSE:", val_mse)


In [ ]:
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error


# from sklearn.ensemble import GradientBoostingRegressor

# kf = KFold(n_splits=5, shuffle=True)

# for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
#     print(f"FOLD {fold+1}")
#     print("--------------------------------")
    
#     # Split data
#     train_fold_data = train_data[train_idx].reshape(-1,train_data.shape[1]*train_data.shape[2])
#     train_fold_labels = train_labels[train_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2])
#     val_fold_data = train_data[val_idx].reshape(-1,train_data.shape[1]*train_data.shape[2])
#     val_fold_labels = train_labels[val_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2])

#     # Model

#     for X, y_true in zip(train_fold_data,train_fold_labels):

#         # X = train_data[0]  # Array with shape (28, 128)
#         # y_true = train_labels[0]  # Vector of length 3

#         # Prepare the data with lagged features
#         window_size = 20
#         future_steps = 3

#         X_lagged = []
#         y_lagged = []
#         for i in range(len(X) - window_size - future_steps + 1):
#             X_lagged.append(X[i:i+window_size])
#             y_lagged.append(X[i+window_size:i+window_size+future_steps])

#         X_lagged = np.array(X_lagged)
#         y_lagged = np.array(y_lagged)


#         # Create DMatrix for XGBoost
#         dtrain = xgb.DMatrix(X_lagged, label=y_lagged)

#         # Define XGBoost parameters
#         params = {
#             'objective': 'reg:squarederror',
#             'eval_metric': 'rmse',
#         }

#         # Train the model
#         num_boost_round = 100
#         bst = xgb.train(params, dtrain, num_boost_round)

#         last_window = X[-window_size:].flatten()
#         dfuture = xgb.DMatrix(np.array([last_window]))
#         future_pred = bst.predict(dfuture)


#         print(mean_squared_error(future_pred.flatten(),y_true.flatten()))


In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor

# kf = KFold(n_splits=5, shuffle=True)

# for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
#     print(f"FOLD {fold+1}")
#     print("--------------------------------")
    
#     # Split data
#     train_fold_data = torch.Tensor(train_data[train_idx].reshape(-1,train_data.shape[1]*train_data.shape[2]))
#     train_fold_labels = torch.Tensor(train_labels[train_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2]))
#     val_fold_data = torch.Tensor(train_data[val_idx].reshape(-1,train_data.shape[1]*train_data.shape[2]))
#     val_fold_labels = torch.Tensor(train_labels[val_idx].reshape(-1,train_labels.shape[1]*train_labels.shape[2]))

#     # Model
#     lr = GradientBoostingRegressor()
#     lr.fit(train_fold_data, train_fold_labels)
#     predictions_train = lr.predict(train_fold_data)
#     predictions_val = lr.predict(val_fold_data)
#     print(mean_squared_error(train_fold_labels,predictions_train))
#     print(mean_squared_error(val_fold_labels,predictions_val))



In [ ]:
# import numpy as np
# import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error

# # Example data (replace with your actual data)
# X = np.random.rand(28, 128)  # 28 time steps, 128 features
# Y = np.random.rand(3, 128)   # 3 time steps to forecast, 128 features

# # Flatten the data
# X_flattened = X.reshape(X.shape[0] * X.shape[1], -1)
# Y_flattened = Y.flatten()

# # Split data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X_flattened[:,0], Y_flattened, test_size=0.2, random_state=42)

# # Create DMatrix for XGBoost
# dtrain = xgb.DMatrix(X_train, label=y_train)
# dtest = xgb.DMatrix(X_test, label=y_test)

# # Define XGBoost parameters
# params = {
#     'objective': 'reg:squarederror',
#     'eval_metric': 'rmse',
# }

# # Train the model
# num_boost_round = 100
# bst = xgb.train(params, dtrain, num_boost_round)

# # Make predictions
# y_pred = bst.predict(dtest)

# # Calculate and print RMSE
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f'RMSE: {rmse}')

# # Predict the next 3 time steps
# # Note: The following is a simplified example, adapt as needed for your use case
# X_future = X[-3:]  # Use the last 3 observations as input for the forecast
# X_future_flattened = X_future.flatten().reshape(1, -1)
# dfuture = xgb.DMatrix(X_future_flattened)
# future_pred = bst.predict(dfuture)
# future_pred_reshaped = future_pred.reshape(3, 128)

# print("Future predictions:")
# print(future_pred_reshaped)


In [60]:
# class Chomp1d(nn.Module):
#     def __init__(self, chomp_size):
#         super(Chomp1d, self).__init__()
#         self.chomp_size = chomp_size

#     def forward(self, x):
#         return x[:, :, :-self.chomp_size].contiguous()

# class TemporalBlock(nn.Module):
#     def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
#         super(TemporalBlock, self).__init__()
#         self.conv1 = nn.Conv1d(n_inputs, n_outputs, kernel_size, stride=stride,
#                                padding=padding, dilation=dilation)
#         self.chomp1 = Chomp1d(padding)
#         self.relu1 = nn.ReLU()
#         self.dropout1 = nn.Dropout(dropout)
#         self.conv2 = nn.Conv1d(n_outputs, n_outputs, kernel_size, stride=stride,
#                                padding=padding, dilation=dilation)
#         self.chomp2 = Chomp1d(padding)
#         self.relu2 = nn.ReLU()
#         self.dropout2 = nn.Dropout(dropout)
        
#         self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
#                                  self.conv2, self.chomp2, self.relu2, self.dropout2)
#         self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
#         self.relu = nn.ReLU()
#         self.init_weights()

#     def init_weights(self):
#         self.conv1.weight.data.normal_(0, 0.01)
#         self.conv2.weight.data.normal_(0, 0.01)
#         if self.downsample is not None:
#             self.downsample.weight.data.normal_(0, 0.01)

#     def forward(self, x):
#         out = self.net(x)
#         res = x if self.downsample is None else self.downsample(x)
#         return self.relu(out + res)

# class TemporalConvNet(nn.Module):
#     def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
#         super(TemporalConvNet, self).__init__()
#         layers = []
#         num_levels = len(num_channels)
#         for i in range(num_levels):
#             dilation_size = 2 ** i
#             in_channels = num_inputs if i == 0 else num_channels[i-1]
#             out_channels = num_channels[i]
#             layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
#                                      padding=(kernel_size-1) * dilation_size, dropout=dropout)]

#         self.network = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.network(x)

# class TCNModel(nn.Module):
#     def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_pred_steps):
#         super(TCNModel, self).__init__()
#         self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
#         self.linear = nn.Linear(num_channels[-1], output_size * num_pred_steps)
#         self.num_pred_steps = num_pred_steps
#         self.output_size = output_size

#     def forward(self, x):
#         y1 = self.tcn(x.transpose(1, 2))
#         o = self.linear(y1[:, :, -1])
#         return o.view(-1, self.num_pred_steps, self.output_size)

# # Hyperparameters
# input_size = 128
# output_size = 128
# num_channels = [128, 256, 128]
# kernel_size = 3
# dropout = 0.2
# num_epochs = 50
# learning_rate = 0.001
# batch_size = 128
# k_folds = 2
# num_pred_steps = 3  # Predicting the next 6 time steps


In [61]:
# def test_tcn_model():
#     model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout, num_pred_steps)
#     dummy_input = torch.randn(10, 25, 128)  # (batch_size, timesteps, features)
#     output = model(dummy_input)
#     assert output.shape == (10, num_pred_steps, output_size), f"Expected output shape (10, {num_pred_steps}, {output_size}), but got {output.shape}"

# def test_tcn_training_step():
#     model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout, num_pred_steps)
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)
#     dummy_input = torch.randn(10, 25, 128)  # (batch_size, timesteps, features)
#     dummy_target = torch.randn(10, num_pred_steps, output_size)  # (batch_size, num_pred_steps, output_size)
#     model.train()
#     outputs = model(dummy_input)
#     loss = criterion(outputs, dummy_target)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     assert loss.item() > 0, "Loss should be greater than 0 after one training step"

# # Run unit tests
# test_tcn_model()
# test_tcn_training_step()
# print("Unit tests passed!")


In [62]:
# # K-Fold Cross Validation
# kf = KFold(n_splits=k_folds, shuffle=True)

# for fold, (train_idx, val_idx) in enumerate(kf.split(train_data)):
#     print(f"FOLD {fold+1}")
#     print("--------------------------------")
    
#     # Split data
#     train_fold_data = torch.Tensor(train_data[train_idx])
#     train_fold_labels = torch.Tensor(train_labels[train_idx])
#     val_fold_data = torch.Tensor(train_data[val_idx])
#     val_fold_labels = torch.Tensor(train_labels[val_idx])

#     # Create DataLoader
#     train_dataset = TensorDataset(train_fold_data, train_fold_labels)
#     val_dataset = TensorDataset(val_fold_data, val_fold_labels)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#     val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#     # Model, loss, optimizer
#     model = TCNModel(input_size, output_size, num_channels, kernel_size, dropout, num_pred_steps).to(train_fold_data.device)
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     # Training loop
#     for epoch in range(num_epochs):
#         model.train()
#         train_loss = 0.0
#         for inputs, targets in train_loader:
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
#             train_loss += loss.item() * inputs.size(0)

#         train_loss /= len(train_loader.dataset)

#         # Validation step
#         model.eval()
#         val_loss = 0.0
#         with torch.no_grad():
#             for inputs, targets in val_loader:
#                 outputs = model(inputs)
#                 loss = criterion(outputs, targets)
#                 val_loss += loss.item() * inputs.size(0)

#         val_loss /= len(val_loader.dataset)

#         if (epoch+1) % 10 == 0:
#             print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

#     print("--------------------------------")


In [ ]:
# ngram_abstracts = np.load("files/ngram_abstracts.npy", mmap_mode="r")
# concept_arr = np.unique(np.load("files/overlapping_concepts.npy"))
# year_arr = np.load("files/year_arr.npy", mmap_mode="r")
# month_arr = np.load("files/month_arr.npy", mmap_mode="r")

In [ ]:
# def compute_word_count_subset(corpus, subset_words):
    
#     for document in tqdm(corpus):
#         for word in document:
#             if word in subset_words:
#                 subset_words[word] += 1
#     return subset_words

# # Compute word count for the subset of words 
# word_count_subset = compute_word_count_subset([row.split() for row in ngram_abstracts], {k:0 for k in np.unique(concept_arr)})

# def filter_dict_by_occurrence(word_count_dict, n):
#     return {word: count for word, count in word_count_dict.items() if count > n}

# filtered_concept_dict = np.array(list(filter_dict_by_occurrence(word_count_subset, 10).keys()))

In [ ]:
# df_ab = pd.DataFrame(data=ngram_abstracts,    # values
#                 columns=["ab"])  # 1st row as the column names
# df_ab["year"] = year_arr
# df_ab["month_arr"] = month_arr

In [ ]:
# df_ab.head()

First pass: Find all occurances of concepts individually & save them in a sfdict format.

In [ ]:
# c_sfdict = sfdict(filename='files/concept_year_dict_filled_1.db') 
# c_dict = c_sfdict.to_dict_nested()
# c_sfdict.close()  # should always close a db

In [ ]:
# c_dict["ab_initio"]["1994"].shape
# np.unique(year_arr)

In [ ]:
# c_encoding_arr = np.zeros((len(c_dict),31,128))
# c_inx_arr = []
# for cnt, (k,v) in enumerate(c_dict.items()):
#     c_encoding_arr[cnt] = np.array([v[str(i)] for i in np.unique(year_arr)])
#     c_inx_arr.append(k) 
# c_inx_arr = np.array(c_inx_arr)
# # c_encoding_arr = np.reshape(c_encoding_arr,(31,len(c_dict),128))

In [ ]:
# np.save("c_encoding_arr",c_encoding_arr)
# np.save("c_inx_arr",c_inx_arr)

In [63]:
# from kerastuner import HyperModel, RandomSearch

In [64]:
# import numpy as np
# import tensorflow as tf
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, RepeatVector, TimeDistributed, Input, BatchNormalization, LayerNormalization
# from tensorflow.keras.models import Model
# from keras_tuner import HyperModel, RandomSearch
# from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
# from tensorflow.keras.regularizers import l2

# # Configuration for window sizes

# bs = 128
# # Convert the NumPy arrays to TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(buffer_size=10000).batch(bs)
# # val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(bs)
# # test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(bs)

# class FeedBack(tf.keras.Model):
#     def __init__(self, units, out_steps):
#         super().__init__()
#         self.out_steps = out_steps
#         self.units = units
#         self.lstm_cell = tf.keras.layers.LSTMCell(units, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.001))
#         self.lstm_cell2 = tf.keras.layers.LSTMCell(units, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.001))
        
#         self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
#         self.dense = tf.keras.layers.Dense(128)
#         self.layer_norm = LayerNormalization()
#         self.dense_warmup = TimeDistributed(Dense(128))

#     def warmup(self, inputs):
#         inputs = self.dense_warmup(inputs)
#         x, *state = self.lstm_rnn(inputs)
#         x = self.layer_norm(x)
#         prediction = self.dense(x)
#         return prediction, state

#     def call(self, inputs, training=None):
#         predictions = []
#         prediction, state = self.warmup(inputs)
#         predictions.append(prediction)

#         for n in range(1, self.out_steps):
#             x = prediction
#             x, state = self.lstm_cell(x, states=state, training=training)
#             x = self.layer_norm(x)
#             x, state = self.lstm_cell2(x, states=state, training=training)
#             x = self.layer_norm(x)
#             prediction = self.dense(x)
#             predictions.append(prediction)

#         predictions = tf.stack(predictions)
#         predictions = tf.transpose(predictions, [1, 0, 2])
#         return predictions

# multi_dense_model = tf.keras.Sequential([
#     # Take the last time step.
#     # Shape [batch, time, features] => [batch, 1, features]
#     tf.keras.layers.Lambda(lambda x: x[:, -1:, :]),
#     # Shape => [batch, 1, dense_units]
#     tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=l2(0.0001)),
#     # tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=l2(0.0001)),
#     tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=l2(0.0001)),
#     # tf.keras.layers.Dropout(0.1),
#     tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=l2(0.0001)),
#     # tf.keras.layers.Dropout(0.25),
#     # Shape => [batch, out_steps*features]
#     tf.keras.layers.Dense(output_window_size*128,
#                           kernel_initializer=tf.initializers.zeros()),
#     # Shape => [batch, out_steps, features]
#     tf.keras.layers.Reshape([output_window_size, 128])
# ])

# CONV_WIDTH = 12
# multi_conv_model = tf.keras.Sequential([
#     # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
#     tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :]),
#     # Shape => [batch, 1, conv_units]
#     tf.keras.layers.Conv1D(384, activation='relu', kernel_size=(CONV_WIDTH)),
#     # tf.keras.layers.Conv1D(1, activation='relu', kernel_size=(CONV_WIDTH//2)),
#     # Shape => [batch, 1,  out_steps*features]
#     tf.keras.layers.Dense(384, activation='relu'),
#     tf.keras.layers.Dense(3*128,
#                           kernel_initializer=tf.initializers.zeros()),
#     # Shape => [batch, out_steps, features]
#     tf.keras.layers.Reshape([3, 128])
# ])

# # Example usage:
# input_window_size = 26
# output_window_size = 3

# model = multi_conv_model#FeedBack(units=128, out_steps=output_window_size)

# model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, min_lr=0)
# early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

# history = model.fit(train_dataset, epochs=100)

# # Evaluate the model on the test data
# test_loss = model.evaluate(train_dataset)
# print(f"Test Loss: {test_loss}")


In [65]:
# def create_tf_dataset(
#     data_array: np.ndarray,
#     input_sequence_length: int,
#     forecast_horizon: int,
#     batch_size: int = 128,
#     shuffle=False,
#     multi_horizon=True,
# ):
#     """Creates tensorflow dataset from numpy array.

#     This function creates a dataset where each element is a tuple `(inputs, targets)`.
#     `inputs` is a Tensor
#     of shape `(batch_size, input_sequence_length, num_routes, 1)` containing
#     the `input_sequence_length` past values of the timeseries for each node.
#     `targets` is a Tensor of shape `(batch_size, forecast_horizon, num_routes)`
#     containing the `forecast_horizon`
#     future values of the timeseries for each node.

#     Args:
#         data_array: np.ndarray with shape `(num_time_steps, num_routes)`
#         input_sequence_length: Length of the input sequence (in number of timesteps).
#         forecast_horizon: If `multi_horizon=True`, the target will be the values of the timeseries for 1 to
#             `forecast_horizon` timesteps ahead. If `multi_horizon=False`, the target will be the value of the
#             timeseries `forecast_horizon` steps ahead (only one value).
#         batch_size: Number of timeseries samples in each batch.
#         shuffle: Whether to shuffle output samples, or instead draw them in chronological order.
#         multi_horizon: See `forecast_horizon`.

#     Returns:
#         A tf.data.Dataset instance.
#     """

#     inputs = tf.keras.utils.timeseries_dataset_from_array(
#         data_array[:-forecast_horizon],
#         None,
#         sequence_length=input_sequence_length,
#         shuffle=False,
#         batch_size=batch_size,
#     )

#     target_offset = (
#         input_sequence_length
#         if multi_horizon
#         else input_sequence_length + forecast_horizon - 1
#     )
#     target_seq_length = forecast_horizon if multi_horizon else 1
#     targets = tf.keras.utils.timeseries_dataset_from_array(
#         data_array[target_offset:],
#         None,
#         sequence_length=target_seq_length,
#         shuffle=False,
#         batch_size=batch_size,
#     )

#     dataset = tf.data.Dataset.zip((inputs, targets))
#     if shuffle:
#         dataset = dataset.shuffle(100)

#     return dataset.prefetch(16).cache()

# def get_encoder(net_config, activation_function, data_shape, latent_dim, batch_size):  
#     encoder_inputs = tf.keras.Input(shape=(data_shape), batch_size=batch_size)
#     x = encoder_inputs
#     for inx in net_config:
#         x = tf.keras.layers.Dense(inx, activation=activation_function)(x)
#     encoder_outputs = tf.keras.layers.Dense(latent_dim)(x)
    
#     return tf.keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

# def get_decoder(net_config, activation_function, data_shape, latent_dim, batch_size):

#     latent_inputs = tf.keras.Input(shape=(data_shape[0],latent_dim), batch_size=(batch_size))
#     x = latent_inputs
#     for inx in net_config[::-1]:
#         x = tf.keras.layers.Dense(inx, activation=activation_function)(x)
    
#     decoder_outputs = tf.keras.layers.Dense(data_shape[1])(x)
    
#     return tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")



# def get_predecoder(data_shape, latent_dim, batch_size):

#     x_predecoder_inputs = tf.keras.Input((data_shape[0]//2,latent_dim),batch_size=(batch_size))

#     # Shape [batch, time, features] => [batch, out_steps, features].
#     # Adding more `lstm_units` just overfits more quickly.
#     x = tf.keras.layers.GRU(80, return_sequences=True)(x_predecoder_inputs)
#     # x = tf.keras.layers.GRU(32, return_sequences=True)(x)
#     # x = tf.keras.layers.GRU(32, return_sequences=True)(x)
#     x = tf.keras.layers.GRU(80, return_sequences=False)(x)
#     # Shape => [batch, out_steps*features].
#     x = tf.keras.layers.Dense(data_shape[0]//2*latent_dim,
#                           kernel_initializer=tf.initializers.zeros())(x)
    
#     x = tf.keras.layers.Reshape([data_shape[0]//2, latent_dim])(x)
#     # Shape => [batch, out_steps, features].
#     return tf.keras.Model(x_predecoder_inputs, [x,x,x], name="xpredecoder")

# # Only does signal processing and no actual time evo in its latent space

# class TAE(tf.keras.Model):
#     def __init__(self, encoder, decoder, pre_decoder, data_shape, **kwargs):
#         super(TAE, self).__init__(**kwargs)
#         self.encoder = encoder
#         self.decoder = decoder
#         self.pre_decoder = pre_decoder
#         self.gamma = 0 
#         self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
#         self.data_shape = data_shape
#         self.latent_dim = 2
        

# @tf.function
# def train_step(model, data,gamma):
#     with tf.GradientTape() as tape:

#         y, _ = data 
#         x = model.encoder(y)

#         x_1, x_2 = tf.split(x,num_or_size_splits=2,axis=1)

#         _, _, x_hat = model.pre_decoder(x_1)

#         x_conc = tf.concat([x_1, x_hat], axis=1)

#         seq_loss = tf.reduce_mean(
#             tf.reduce_sum(tf.keras.losses.mean_squared_error(x_2, x_hat), axis=-1))        
#         seq_loss = seq_loss/(model.latent_dim*(model.data_shape[0]//2))
        
#         y_hat = model.decoder(x_conc) 
#         reconstruction_loss = tf.reduce_mean(
#             tf.reduce_sum(tf.keras.losses.mean_absolute_error(y, y_hat), axis=-1))
#         reconstruction_loss = reconstruction_loss/(model.data_shape[0]*model.data_shape[1])
            
#         total_loss = reconstruction_loss + gamma*seq_loss
    
#     grads = tape.gradient(total_loss, model.trainable_weights)
#     model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

#     return total_loss, reconstruction_loss, seq_loss

# def gamma_sched(n_iter, start=0.0, stop=0.25,  n_cycle=1, ratio=0.8):
#     L = np.ones(n_iter) * stop
#     period = n_iter/n_cycle
#     step = (stop-start)/(period*ratio) # linear schedule

#     for c in range(n_cycle):
#         v, i = start, 0
#         while v <= stop and (int(i+c*period) < n_iter):
#             L[int(i+c*period)] = v#np.sin(3*i/period)       
#             v += step
#             i += 1
#     return L 

# batch_size = 128
# input_sequence_length = 30
# multi_horizon = True
# n_traj = 128

# train_dataset = (
#     create_tf_dataset(normalized_data, input_sequence_length, 1, batch_size)
# )

# # test_dataset = create_tf_dataset(
# #     test_array,
# #     input_sequence_length,
# #     1,
# #     batch_size=test_array.shape[0],
# #     shuffle=False,
# #     multi_horizon=multi_horizon,
# # )

# data_shape = (input_sequence_length,n_traj)

# latent_dim = 3
# net_config = np.array([300,100])
# activation_function = "leaky_relu"



# encoder = get_encoder(net_config, activation_function, data_shape, latent_dim, batch_size)
# decoder = get_decoder(net_config, activation_function, data_shape, latent_dim, batch_size)
# predecoder = get_predecoder(data_shape, latent_dim, batch_size)

# epochs = 500
# lr_sched = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate=0.0001,
#     decay_steps=epochs*int(len(train_dataset)),
#     alpha=0.00005/0.0001,
#     )
# loss_arr = np.zeros(epochs)
# rc_loss_arr =  np.zeros(epochs)
# seq_loss_arr = np.zeros(epochs)

# ae_model = TAE(encoder, decoder, predecoder, data_shape)
# # ae_model.gamma = 0.5
# # ae_model.latent_dim = 2
# gamma_arr = gamma_sched(epochs,stop=0.15,ratio=0.75)
# # ae_model.optimizer = tf.keras.optimizers.Adam(learning_rate=lr_sched)
# # beta_arr = beta_sched(epochs,stop=0.15,ratio=0.5)

# for epoch in tqdm(range(epochs)):
#     gamma = gamma_arr[epoch] 
    
#     for step, batch_data in enumerate(train_dataset):
#         loss, rc_loss, seq_loss = train_step(ae_model, batch_data,np.float32(gamma))

#         loss_arr[epoch] = loss
#         rc_loss_arr[epoch] = rc_loss
#         seq_loss_arr[epoch] = seq_loss


# loss_arr[-1], rc_loss_arr[-1], seq_loss_arr[-1]



In [ ]:
# import numpy as np
# import tensorflow as tf
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, RepeatVector, TimeDistributed, Input, BatchNormalization, LayerNormalization
# from tensorflow.keras.models import Model
# from keras_tuner import HyperModel, RandomSearch
# from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
# from tensorflow.keras.regularizers import l2

# # Configuration for window sizes
# input_window_size = 26  # Number of input time steps (e.g., 0-25)
# output_window_size = 3  # Number of time steps to predict (e.g., 25-28)

# # Assuming your data is in a NumPy array of shape (10000, 31, 128)
# data = c_encoding_arr

# # Ensure the total number of time steps is enough to create the windows
# total_time_steps = data.shape[1]
# assert total_time_steps >= input_window_size + output_window_size, "Insufficient time steps in data."

# # Normalize the data
# scaler = StandardScaler()

# # Reshape data to 2D for normalization
# reshaped_data = data.reshape(-1, data.shape[-1])  # Shape: (10000*31, 128)
# normalized_data = scaler.fit_transform(reshaped_data)

# # Reshape back to original shape
# normalized_data = normalized_data.reshape(data.shape)

# # Prepare training inputs and outputs
# X_train = normalized_data[:, :input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_train = normalized_data[:, input_window_size:input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)

# # Prepare validation inputs and outputs
# X_val = normalized_data[:, 1:1 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_val = normalized_data[:, 1 + input_window_size:1 + input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)

# # Prepare test inputs and outputs
# X_test = normalized_data[:, 2:2 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_test = normalized_data[:, 2 + input_window_size:2 + input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)

# bs = 64
# # Convert the NumPy arrays to TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=10000).batch(bs)
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(bs)
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(bs)

# class StackedLSTM(tf.keras.Model):
#     def __init__(self, units, num_layers, out_steps):
#         super().__init__()
#         self.out_steps = out_steps
#         self.units = units
#         self.num_layers = num_layers

#         # Create a list of LSTM cells
#         self.lstm_cells = [tf.keras.layers.LSTMCell(units, dropout=0.2, recurrent_dropout=0.2, kernel_regularizer=l2(0.001)) for _ in range(num_layers)]
#         self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cells, return_state=True)
        
#         self.dense = tf.keras.layers.Dense(128)
#         self.layer_norm = LayerNormalization()
#         self.dense_warmup = TimeDistributed(Dense(128))

#     def warmup(self, inputs):
#         inputs = self.dense_warmup(inputs)
#         x, *state = self.lstm_rnn(inputs)
#         x = self.layer_norm(x)
#         prediction = self.dense(x)
#         return prediction, state

#     def call(self, inputs, training=None):
#         predictions = []
#         prediction, state = self.warmup(inputs)
#         predictions.append(prediction)

#         for n in range(1, self.out_steps):
#             x = prediction
#             x, state = self.lstm_cells[-1](x, states=state, training=training)
#             x = self.layer_norm(x)
#             prediction = self.dense(x)
#             predictions.append(prediction)

#         predictions = tf.stack(predictions)
#         predictions = tf.transpose(predictions, [1, 0, 2])
#         return predictions

# # Example usage:
# input_window_size = 26
# output_window_size = 3
# num_layers = 3  # Number of stacked LSTM layers

# model = StackedLSTM(units=128, num_layers=num_layers, out_steps=output_window_size)

# model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))

# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1, min_lr=0)
# early_stop = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

# history = model.fit(train_dataset, epochs=300, validation_data=val_dataset, callbacks=[reduce_lr, early_stop])

# # Evaluate the model on the test data
# test_loss = model.evaluate(test_dataset)
# print(f"Test Loss: {test_loss}")


In [ ]:
# import numpy as np
# import tensorflow as tf
# from sklearn.preprocessing import StandardScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional, RepeatVector, TimeDistributed, Input, BatchNormalization
# from tensorflow.keras.models import Model
# from keras_tuner import HyperModel, RandomSearch
# from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau,EarlyStopping
# from tensorflow.keras.regularizers import l2
# from tensorflow.keras.layers import GRUCell

# # Configuration for window sizes
# input_window_size = 26  # Number of input time steps (e.g., 0-25)
# output_window_size = 3  # Number of time steps to predict (e.g., 25-28)

# # Assuming your data is in a NumPy array of shape (10000, 31, 128)
# data = c_encoding_arr

# # Ensure the total number of time steps is enough to create the windows
# total_time_steps = data.shape[1]
# assert total_time_steps >= input_window_size + output_window_size, "Insufficient time steps in data."

# # Normalize the data
# scaler = StandardScaler()

# # Reshape data to 2D for normalization
# reshaped_data = data.reshape(-1, data.shape[-1])  # Shape: (10000*31, 128)
# normalized_data = scaler.fit_transform(reshaped_data)

# # Reshape back to original shape
# normalized_data = normalized_data.reshape(data.shape)

# # Prepare training inputs and outputs
# X_train = normalized_data[:, :input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_train = normalized_data[:, input_window_size:input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)

# # Prepare validation inputs and outputs
# X_val = normalized_data[:, 1:1 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_val = normalized_data[:, 1 + input_window_size:1 + input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)

# # Prepare test inputs and outputs
# X_test = normalized_data[:, 2:2 + input_window_size, :]  # Shape: (10000, input_window_size, 128)
# Y_test = normalized_data[:, 2 + input_window_size:2 + input_window_size + output_window_size, :]  # Shape: (10000, output_window_size, 128)


# bs = 64
# # Convert the NumPy arrays to TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(buffer_size=10000).batch(bs)
# val_dataset = tf.data.Dataset.from_tensor_slices((X_val, Y_val)).batch(bs)
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(bs)



# class FeedBack(tf.keras.Model):
#     def __init__(self, units, out_steps):
#         super().__init__()
#         self.out_steps = out_steps
#         self.units = units
#         self.lstm_cell = tf.keras.layers.LSTMCell(units,dropout=0.2,recurrent_dropout=0.2, kernel_regularizer=l2(0.001))
#         # self.lstm_cell = tf.keras.layers.GRUCell(units,dropout=0.2,recurrent_dropout=0.2, kernel_regularizer=l2(0.001))

#         # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
#         self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
#         self.dense = tf.keras.layers.Dense(128)
#         self.batch_norm = BatchNormalization()
#         self.dense_warmup = TimeDistributed(Dense(128))


#     def warmup(self, inputs):
#         # inputs.shape => (batch, time, features)
#         # x.shape => (batch, lstm_units)
#         inputs = self.dense_warmup(inputs)
#         x, *state = self.lstm_rnn(inputs)
#         x = self.batch_norm(x)
#         # predictions.shape => (batch, features)
#         prediction = self.dense(x)
#         return prediction, state

#     def call(self, inputs, training=None):
#         # Use a TensorArray to capture dynamically unrolled outputs.
#         predictions = []
#         # Initialize the LSTM state.
#         prediction, state = self.warmup(inputs)

#         # Insert the first prediction.
#         predictions.append(prediction)

#         # Run the rest of the prediction steps.
#         for n in range(1, self.out_steps):
#             # Use the last prediction as input.
#             x = prediction
#             # Execute one lstm step.
#             x, state = self.lstm_cell(x, states=state,
#                                     training=training)
#             x = self.batch_norm(x)
#             # Convert the lstm output to a prediction.
#             prediction = self.dense(x)
#             # Add the prediction to the output.
#             predictions.append(prediction)

#         # predictions.shape => (time, batch, features)
#         predictions = tf.stack(predictions)
#         # predictions.shape => (batch, time, features)
#         predictions = tf.transpose(predictions, [1, 0, 2])
#         return predictions



# # Example usage:
# # Assuming input_window_size is defined as 26 and input features as 128.
# input_window_size = 26
# output_window_size = 3

# model = FeedBack(units=128, out_steps=output_window_size)


# model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001))

# reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
#                               patience=10, verbose=1,min_lr=0)


# early_stop= EarlyStopping(monitor='loss', patience=20, verbose=1)

# history = model.fit(train_dataset,
#           epochs=300,
#           validation_data=val_dataset,
#           callbacks=[reduce_lr,early_stop])


# # Evaluate the model on the test data
# test_loss = model.evaluate(test_dataset)
# print(f"Test Loss: {test_loss}")


In [ ]:
# model = Sequential([
#     LSTM(128, activation='relu', input_shape=(input_window_size, 128),return_sequences=True,dropout=0.3),
#     LSTM(128, activation='relu',dropout=0.3),
#     RepeatVector(output_window_size),
#     TimeDistributed(Dense(3))
# ])
# model.summary()

In [67]:
# i = 4
# predictions = model.predict(X_test)
# inx = c_inx_arr

# # Inverse transform the scaled predictions
# original_predictions = scaler.inverse_transform(predictions.reshape(-1, 128)).reshape(predictions.shape)
# goal_predictions = scaler.inverse_transform(Y_test.reshape(-1, 128)).reshape(Y_test.shape)

# def similarity_cosine(vec1, vec2):
#     cosine_similarity = np.dot(vec1, vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))
#     return cosine_similarity

In [66]:
# p_arr = []
# g_arr = []

# i1 = np.random.randint(len(inx))
# i2 = np.random.randint(len(inx))

# for i in [0,1,2]:
#     p_arr.append(similarity_cosine(original_predictions[i1,i],original_predictions[i2,i]))
#     g_arr.append(similarity_cosine(goal_predictions[i1,i],goal_predictions[i2,i]))

# plt.plot([1,2,3],p_arr,label="Prediction")
# plt.plot([1,2,3],g_arr,label="True")
# plt.gca().xaxis.set_major_locator(plt.MaxNLocator(integer=True))
# plt.xlabel("Future Time Steps")
# plt.ylabel("Cosine Similarity")
# plt.legend()
# plt.title(str(inx[i1])+" "+str(inx[i2]))

In [ ]:

# class TimeSeriesHyperModel(HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(Bidirectional(LSTM(units=hp.Int('units_1', min_value=32, max_value=128, step=32), 
#                                     activation='relu', return_sequences=True), 
#                                 input_shape=(input_window_size, 128)))
#         model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
#         model.add(Bidirectional(LSTM(units=hp.Int('units_2', min_value=32, max_value=128, step=32), 
#                                     activation='relu', return_sequences=False)))
#         model.add(RepeatVector(output_window_size))
#         model.add(Bidirectional(LSTM(units=hp.Int('units_3', min_value=32, max_value=128, step=32), 
#                                     activation='relu', return_sequences=True)))
#         model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
#         model.add(TimeDistributed(Dense(128)))
        
#         model.compile(optimizer='adam', loss='mse')
#         return model

# class TimeSeriesHyperModel(HyperModel):
#     def build(self, hp):
#         model = Sequential()
#         model.add(LSTM(units=hp.Int('units_1', min_value=32, max_value=128, step=32), 
#                                     activation='relu',input_shape=(input_window_size, 128)))
#         model.add(RepeatVector(output_window_size))
#         model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))
#         model.add(LSTM(units=hp.Int('units_3', min_value=32, max_value=128, step=32), 
#                                     activation='relu', return_sequences=True))
#         model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
#         model.add(TimeDistributed(Dense(128)))
        
#         model.compile(optimizer='adam', loss='mse')
#         return model


# # Initialize the tuner
# tuner = RandomSearch(
#     TimeSeriesHyperModel(),
#     objective='val_loss',
#     max_trials=20,
#     executions_per_trial=2,
#     directory='hyperparameter_tuning',
#     project_name='time_series_prediction'
# )

# Perform the search
# tuner.search(train_dataset, epochs=20, validation_data=val_dataset)

# Retrieve the best model
# best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model on the full dataset
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(32)
# best_model.fit(train_dataset, epochs=20, validation_data=val_dataset)

# Evaluate the best model on the test data
# test_loss = best_model.evaluate(test_dataset)
# print(f"Test Loss: {test_loss}")

In [68]:
# def get_encoder(net_config, activation_function, data_shape, latent_dim, batch_size):  
#     encoder_inputs = tf.keras.Input(shape=(data_shape), batch_size=batch_size)
#     x = encoder_inputs
#     for inx in net_config:
#         x = tf.keras.layers.Dense(inx, activation=activation_function)(x)
#     encoder_outputs = tf.keras.layers.Dense(latent_dim)(x)
#     return tf.keras.Model(encoder_inputs, encoder_outputs, name = "encoder")

# def get_decoder(net_config, activation_function, data_shape, latent_dim, batch_size):
#     latent_inputs = tf.keras.Input(shape=(data_shape[0],latent_dim), batch_size=(batch_size))
#     x = latent_inputs
#     for inx in net_config[::-1]:
#         x = tf.keras.layers.Dense(inx, activation=activation_function)(x)
#     decoder_outputs = tf.keras.layers.Dense(data_shape[1])(x)
#     return tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")



# def get_predecoder(data_shape, latent_dim, batch_size):

#     x_predecoder_inputs = tf.keras.Input((data_shape[0]//2,latent_dim),batch_size=(batch_size))

#     # Shape [batch, time, features] => [batch, out_steps, features].
#     # Adding more `lstm_units` just overfits more quickly.
#     x = tf.keras.layers.GRU(80, return_sequences=True)(x_predecoder_inputs)
#     # x = tf.keras.layers.GRU(32, return_sequences=True)(x)
#     # x = tf.keras.layers.GRU(32, return_sequences=True)(x)
#     x = tf.keras.layers.GRU(80, return_sequences=False)(x)
#     # Shape => [batch, out_steps*features].
#     x = tf.keras.layers.Dense(data_shape[0]//2*latent_dim,
#                           kernel_initializer=tf.initializers.zeros())(x)
    
#     x = tf.keras.layers.Reshape([data_shape[0]//2, latent_dim])(x)
#     # Shape => [batch, out_steps, features].
#     return tf.keras.Model(x_predecoder_inputs, [x,x,x], name="xpredecoder")